In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import shlex
import nltk
%matplotlib inline

In [2]:
f = open('MLK.txt', 'r')
quotes = []
for line in f:
    if line != '\n':
        line = line.strip('\n')
        quotes.append(line)

In [3]:
len(quotes)

267

In [4]:
#word counts
word_count = {}
for quote in quotes:
    words = quote.split()
    for word in words:
        if word not in word_count.keys():
            word_count[word] = 0
        word_count[word] += 1
word_count

{'You': 9,
 'can': 39,
 'kill': 3,
 'the': 361,
 'dreamer,': 2,
 'but': 53,
 'you': 48,
 'can’t': 5,
 'dream.': 3,
 'A': 21,
 'man': 22,
 'ride': 1,
 'your': 9,
 'back': 3,
 'unless': 1,
 'it’s': 2,
 'bent.': 1,
 'Our': 3,
 'lives': 2,
 'begin': 2,
 'to': 194,
 'end': 3,
 'day': 8,
 'we': 49,
 'become': 14,
 'silent': 1,
 'about': 5,
 'things': 2,
 'that': 118,
 'matter.': 1,
 'When': 8,
 'are': 51,
 'right': 9,
 'cannot': 19,
 'be': 55,
 'too': 5,
 'radical;': 1,
 'when': 18,
 'wrong,': 1,
 'conservative.': 1,
 'True': 2,
 'peace': 10,
 'is': 172,
 'not': 71,
 'merely': 7,
 'absence': 2,
 'of': 259,
 'tension:': 1,
 'it': 36,
 'presence': 1,
 'justice.': 5,
 'I': 59,
 'want': 7,
 'white': 7,
 'man’s': 3,
 'brother,': 1,
 'his': 16,
 'brother-in-law.': 1,
 'Darkness': 5,
 'only': 22,
 'driven': 1,
 'out': 15,
 'with': 48,
 'light,': 1,
 'more': 23,
 'darkness.': 1,
 'Power': 2,
 'without': 8,
 'love': 28,
 'reckless': 1,
 'and': 173,
 'abusive,': 1,
 'power': 13,
 'sentimental': 1,
 'a

In [5]:
#Most common words
l = []
for keys, values in list(word_count.items()):
    l.append((values,keys))
l.sort(reverse=True)
l[:5]

[(361, 'the'), (259, 'of'), (194, 'to'), (173, 'and'), (172, 'is')]

In [6]:
import io

PATH = "MLK.txt" 
sequence_length = 40
step = 3


text = []
chars = []


with io.open(PATH, 'r', encoding='utf8') as f:
    text = f.read().lower()
    print(text)
    chars = sorted(list(set(text)))
    print("Chars: ", chars)

you can kill the dreamer, but you can’t kill the dream.
a man can’t ride your back unless it’s bent.
our lives begin to end the day we become silent about things that matter.
when you are right you cannot be too radical; when you are wrong, you cannot be too conservative.
true peace is not merely the absence of tension: it is the presence of justice.
i want to be the white man’s brother, not his brother-in-law.
darkness is only driven out with light, not more darkness.
power without love is reckless and abusive, and love without power is sentimental and anemic.
let no man pull you so low as to hate him.
even though we face the difficulties of today and tomorrow, i still have a dream.
the time is always right to do what is right.
injustice anywhere is a threat to justice everywhere.
human salvation lies in the hands of the creatively maladjusted.
our scientific power has outrun our spiritual power. we have guided missiles and misguided men.
i have a dream that one day this nation will r

### Keras trial

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
import keras.utils as ku

Using TensorFlow backend.


In [8]:
data = """The cat and her kittens
They put on their mittens,
To eat a Christmas pie.
The poor little kittens
They lost their mittens,
And then they began to cry.
O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens."
"If it be so, ye shall not go,
For ye are naughty kittens."""

In [9]:
tokenizer = Tokenizer()

In [10]:

def dataset_preparation(data):

    # basic cleanup
    corpus = data.lower().split("\n")

    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

In [11]:
def create_model(predictors, label, max_sequence_len, total_words):
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences = True))
    # model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
    print(model.summary())
    return model 

In [12]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [16]:
X, Y, maxlen, total_words = dataset_preparation(data)
model = create_model(X, Y, maxlen, total_words)
text = generate_text("cat and", 9, maxlen)
print(text)

Epoch 1/100
48/48 [==============================] - 2s 32ms/step - loss: 3.7624 - acc: 0.0000e+00
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7585 - acc: 0.0417
Epoch 3/100
48/48 [==============================] - 0s 950us/step - loss: 3.7557 - acc: 0.0417
Epoch 4/100
48/48 [==============================] - 0s 899us/step - loss: 3.7518 - acc: 0.0625
Epoch 5/100
48/48 [==============================] - 0s 957us/step - loss: 3.7475 - acc: 0.0625
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7422 - acc: 0.0625
Epoch 7/100
48/48 [==============================] - 0s 925us/step - loss: 3.7328 - acc: 0.0833
Epoch 8/100
48/48 [==============================] - 0s 994us/step - loss: 3.7231 - acc: 0.0625
Epoch 9/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7104 - acc: 0.0625
Epoch 10/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6862 - acc: 0.0625
Epoch 11/100
48/48 [========================

Epoch 87/100
48/48 [==============================] - 0s 1ms/step - loss: 2.4048 - acc: 0.3542
Epoch 88/100
48/48 [==============================] - 0s 823us/step - loss: 2.4049 - acc: 0.3333
Epoch 89/100
48/48 [==============================] - 0s 980us/step - loss: 2.3785 - acc: 0.3542
Epoch 90/100
48/48 [==============================] - 0s 891us/step - loss: 2.3600 - acc: 0.3542
Epoch 91/100
48/48 [==============================] - 0s 975us/step - loss: 2.3535 - acc: 0.3750
Epoch 92/100
48/48 [==============================] - 0s 803us/step - loss: 2.3282 - acc: 0.3542
Epoch 93/100
48/48 [==============================] - 0s 844us/step - loss: 2.2862 - acc: 0.3750
Epoch 94/100
48/48 [==============================] - 0s 918us/step - loss: 2.2886 - acc: 0.3958
Epoch 95/100
48/48 [==============================] - 0s 863us/step - loss: 2.2436 - acc: 0.3958
Epoch 96/100
48/48 [==============================] - 0s 1ms/step - loss: 2.2566 - acc: 0.3750
Epoch 97/100
48/48 [==============

In [ ]:
!git add .
!git commit -m "trial commit"

### PyTorch trial